## Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score, RandomizedSearchCV, GridSearchCV, TunedThresholdClassifierCV, cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, roc_curve, ConfusionMatrixDisplay, precision_recall_curve, average_precision_score, make_scorer, f1_score, precision_score, recall_score
from sklearn.metrics import get_scorer_names, balanced_accuracy_score
import matplotlib.pyplot as plt
from typing_extensions import final
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns

Option 1

In [ ]:
# Patient has visit -> all days are 1
fitbit_data = pd.read_csv('/content/drive/My Drive/colab_data/data_daily_w_visits.csv')

# Loop through each user group
for user_id, group in fitbit_data.groupby('fitbit_user_id'):
    if (group['visit_day'] == 1).any():
        # Set all rows for this user to 1 in the original dataframe
        fitbit_data.loc[fitbit_data['fitbit_user_id'] == user_id, 'visit_day'] = 1

Option 2

In [ ]:
# Load your data
fitbit_data = pd.read_csv('/content/drive/My Drive/colab_data/data_daily_w_visits.csv')

# Iterate by user
for user_id, group in fitbit_data.groupby('fitbit_user_id'):
    visit_days = group[group['visit_day'] == 1]['days'].values

    for visit_day in visit_days:
        # Get range from (visit_day - 14) to (visit_day + 14)
        lower = visit_day - 14
        upper = visit_day + 14

        # Create mask for the affected rows for this user
        mask = (
            (fitbit_data['fitbit_user_id'] == user_id) &
            (fitbit_data['days'] >= lower) &
            (fitbit_data['days'] <= upper)
        )

        fitbit_data.loc[mask, 'visit_day'] = 1

In [ ]:
# Ensure date is datetime and sort
fitbit_data['date'] = pd.to_datetime(fitbit_data['date'])
fitbit_data = fitbit_data.sort_values(by=['fitbit_user_id', 'date'])

In [ ]:
# Organizing columns
measure_cols = ['avgWeight_per_day', 'calories', 'heart', 'steps']
survey_cols = ['diet', 'medication', 'symptoms']
result_col = 'visit_day'

In [ ]:
fitbit_data[result_col] = fitbit_data[result_col].fillna(0)

In [ ]:
def fix_survey_cols(df, cols):
    df = df.copy()

    # Shift survey columns backward by one day within each user group
    for col in cols:
        df[f'{col}_cur'] = df.groupby('fitbit_user_id')[col].shift(-1)

    return df

fitbit_data = fix_survey_cols(fitbit_data, survey_cols)

In [ ]:
survey_cols = ['diet_cur', 'medication_cur', 'symptoms_cur']

In [ ]:
# Clears rows with empty columns to start
def prelim_row_removal(df, cols):
    cleaned_groups = []

    for _, group in df.groupby('fitbit_user_id'):
        group = group.copy()
        to_remove = []

        for idx, row in group.iterrows():
            if row[cols].isna().all() and row['study_group'] != "No App":
                to_remove.append(idx)
            else:
                break  # Stop as soon as we hit a row with any non-NaN value

        group = group.drop(index=to_remove)
        cleaned_groups.append(group)

    return pd.concat(cleaned_groups).reset_index(drop=True)


fitbit_data = prelim_row_removal(fitbit_data, measure_cols+survey_cols)

In [ ]:
def mark_day_for_removal(df, cols, max_nans, window):
    df = df.copy()
    df['remove'] = False

    for _, group in df.groupby('fitbit_user_id'):
        group = group.copy()

        for col in cols:
            n = len(group)
            for start in range(n):
                end = min(start + window, n)
                window_slice = group.iloc[start:end]

                # Count NaNs in this window for this column
                nan_count = window_slice[col].isna().sum()

                if nan_count >= max_nans:
                    # Mark rows with NaN in this column for removal
                    for idx in window_slice.index:
                        if pd.isna(group.at[idx, col]):
                            df.at[idx, 'remove'] = True

    return df

fitbit_data = mark_day_for_removal(fitbit_data, measure_cols+survey_cols, 3, 7)

In [ ]:
def impute_forward_fill(df, cols):
    df = df.copy()
    for _, group in df.groupby('fitbit_user_id'):
        group = group.copy()
        for col in cols:
            group[col] = group[col].ffill()  # forward fill
        df.loc[group.index, cols] = group[cols]
    return df

fb_data = impute_forward_fill(fitbit_data, measure_cols+survey_cols)

In [ ]:
def one_hot_encode_with_nan(df, cols):
    df = df.copy()
    for col in cols:
        dummies = pd.get_dummies(df[col], prefix=col)

        # Set dummy rows to np.nan where original was NaN
        dummies[df[col].isna()] = np.nan

        # Convert dummies to float so XGBoost can handle them
        dummies = dummies.astype(float)

        df = pd.concat([df, dummies], axis=1)
        df.drop(columns=[col], inplace=True)
    return df

fb_data = one_hot_encode_with_nan(fb_data, survey_cols)

In [ ]:
oneh_cols = ['diet_cur_0.0', 'diet_cur_1.0', 'diet_cur_2.0', 'medication_cur_0.0', 'medication_cur_1.0', 'medication_cur_2.0', 'symptoms_cur_0.0', 'symptoms_cur_1.0', 'symptoms_cur_2.0']

In [ ]:
def final_clean(df, cols):
    df = df.copy()

    # Remove rows marked True in the 'remove' column
    df = df[df['remove'] != True]

    # Remove rows that have any NaN in the specified columns
    df = df.dropna(subset=cols)

    return df

fb_data = final_clean(fb_data, measure_cols+oneh_cols)

In [ ]:
scaler = MinMaxScaler().fit(fb_data[measure_cols])
fb_data[measure_cols] = scaler.transform(fb_data[measure_cols])

In [ ]:
fb_data

## KFold Cross Validation

In [ ]:
x = fb_data[measure_cols+oneh_cols+['fitbit_user_id']]
targets = fb_data[result_col].astype(int)

sgkf = StratifiedGroupKFold(n_splits=5)

groups = x['fitbit_user_id']  # or however you track user grouping

In [ ]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
cross_val_score(rf, x, targets, groups=groups, cv=sgkf, scoring="roc_auc")

In [ ]:
cross_val_score(rf, x, targets, groups=groups, cv=sgkf, scoring="average_precision")

In [ ]:
cross_val_score(rf, x, targets, groups=groups, cv=sgkf, scoring="f1")

Tuning Hyperparameters

In [ ]:
# Broad parameter space
param_dist_rf = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf_random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist_rf,
    n_iter=30,
    scoring='roc_auc',
    n_jobs=-1,
    cv=sgkf.split(x, targets, groups=groups),
    verbose=1,
    random_state=42
)

rf_random_search.fit(x, targets, groups=groups)

print("Best params:", rf_random_search.best_params_)
print("Best ROC AUC:", rf_random_search.best_score_)

In [ ]:
param_grid_rf = {
    'n_estimators': [150, 200, 250],
    'max_depth': [5, 10, 15],
    'min_samples_split': [4, 5, 6],
    'min_samples_leaf': [3, 4, 5],
    'max_features': ['sqrt'],
    'bootstrap': [True]
}

rf_grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring='roc_auc',
    n_jobs=-1,
    cv=sgkf.split(x, targets, groups=groups),
    verbose=1
)

rf_grid_search.fit(x, targets, groups=groups)

print("Best RF grid params:", rf_grid_search.best_params_)
print("Best RF grid ROC AUC:", rf_grid_search.best_score_)

ROC Curves + Confusion Matricies

In [ ]:
best_params_week = {'min_samples_split': 5, 'n_estimators': 200, 'min_sample_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}
best_params_user = {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 20, 'bootstrap': True}

In [1]:
def train_evaluate_visualize(X_train, y_train, X_val, y_val, **params):
    model = RandomForestClassifier(random_state=42, n_jobs=-1, **params)
    model.fit(X_train, y_train)

    # Hard predictions
    y_pred_train = model.predict(X_train)
    y_pred_val   = model.predict(X_val)

    # Probabilities for ROC/AUC
    y_prob_train = model.predict_proba(X_train)[:, 1]
    y_prob_val   = model.predict_proba(X_val)[:, 1]

    # Metrics
    train_acc = accuracy_score(y_train, y_pred_train)
    val_acc   = accuracy_score(y_val, y_pred_val)
    train_auc = roc_auc_score(y_train, y_prob_train)
    val_auc   = roc_auc_score(y_val, y_prob_val)

    print(f"Train  Acc: {train_acc:.3f},  Validation Acc: {val_acc:.3f}")
    print(f"Train  AUC: {train_auc:.3f},  Validation AUC: {val_auc:.3f}")

    # Confusion Matrix
    cm = confusion_matrix(y_val, y_pred_val)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap="Blues")
    plt.title("Confusion Matrix")
    plt.show()

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_val, y_prob_val)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC (AUC = {val_auc:.2f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_val, y_prob_val)
    avg_precision = average_precision_score(y_val, y_prob_val)
    plt.figure()
    plt.plot(recall, precision, label=f'PR Curve (AP = {avg_precision:.2f})')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve")
    plt.legend(loc="lower left")
    plt.grid(True)
    plt.show()

    return model, train_acc, val_acc, train_auc, val_auc

In [2]:
modelsSGK = []
for train_idx, val_idx in sgkf.split(x, targets, groups=groups):
    X_tr, y_tr = x.iloc[train_idx], targets.iloc[train_idx]
    X_vl, y_vl = x.iloc[val_idx], targets.iloc[val_idx]

    model, train_acc, val_acc, train_auc, val_auc = train_evaluate_visualize(
        X_tr, y_tr,
        X_vl, y_vl,
        **best_params_user
    )

    modelsSGK.append(model)

NameError: name 'sgkf' is not defined

In [ ]:
best_params = {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}

In [ ]:
# Define model with best parameters
rf_res = RandomForestClassifier(random_state=42, n_jobs=-1, **best_params_user)

# Define scoring metrics
scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']

# Run CV
cv_results = cross_validate(
    rf_res,
    x,
    targets,
    groups=groups,
    cv=sgkf,
    scoring=scoring,
    return_train_score=True
)

In [ ]:
# Convert to DataFrame
cv_df = pd.DataFrame(cv_results)

# Clean summary table
summary = cv_df[[col for col in cv_df.columns if col.startswith('test_')]].describe().T
summary['metric'] = summary.index.str.replace('test_', '')
summary = summary[['metric', 'mean', 'std']]
summary.reset_index(drop=True, inplace=True)

summary

In [ ]:
# Only test metrics
metric_data = {k: v for k, v in cv_results.items() if k.startswith('test_')}
metric_df = pd.DataFrame(metric_data)

# Rename for clarity
metric_df.columns = [col.replace('test_', '') for col in metric_df.columns]
metric_df = metric_df.melt(var_name='Metric', value_name='Score')

# Boxplot
plt.figure(figsize=(8, 5))
sns.boxplot(data=metric_df, x='Metric', y='Score')
plt.title("Cross-Validation Metric Distribution (Test Sets)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
train_test_df = pd.DataFrame({
    'Train Accuracy': cv_results['train_accuracy'],
    'Test Accuracy': cv_results['test_accuracy'],
    'Train AUC': cv_results['train_roc_auc'],
    'Test AUC': cv_results['test_roc_auc']
})

train_test_df.plot(kind='bar', figsize=(10, 5))
plt.title("Train vs Test Scores per Fold")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.grid(True)
plt.tight_layout()
plt.show()


Tuning threshold

In [ ]:
for train_idx, val_idx in sgkf.split(x, targets, groups):
    X_train, X_val = x.iloc[train_idx], x.iloc[val_idx]
    y_train, y_val = targets.iloc[train_idx], targets.iloc[val_idx]
    groups_train = groups.iloc[train_idx]
    break

In [ ]:
rf = RandomForestClassifier(
    **best_params_user,
    random_state=42,
    n_jobs=-1,
)

rf_thresh = TunedThresholdClassifierCV(
    estimator=rf,
    scoring="balanced_accuracy",
    store_cv_results=True,
    cv=StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42).split(
        X_train, y_train, groups_train
    ),
    thresholds = 200,
    refit=True
)

rf_thresh.fit(X_train, y_train)

# --- Best threshold found ---
best_threshold = rf_thresh.best_threshold_
print(f"✅ Tuned threshold: {best_threshold:.3f}")


In [ ]:
plt.plot(
    rf_thresh.cv_results_["thresholds"],
    rf_thresh.cv_results_["scores"],
    label="accuracy"
)
plt.plot(
    rf_thresh.best_threshold_,
    rf_thresh.best_score_,
    "o",
    markersize = 10,
    color ="tab:orange",
    label="Optimal cut off point"
)
plt.legend()


In [ ]:
y_pred_val = rf_thresh.predict(X_val)

cm = confusion_matrix(y_val, y_pred_val)
print("Confusion Matrix on Validation Set:\n", cm)

In [ ]:
# --- Model definition ---
model = RandomForestClassifier(
   **best_params_user,
    random_state=42,
    n_jobs=-1
)

# --- Wrapper for threshold tuning ---
tuned_model = TunedThresholdClassifierCV(
    estimator=model,
    scoring="balanced_accuracy",
    store_cv_results=True,
    thresholds=200,  # granularity of threshold search
    refit=True
)

# --- Cross-validate outer loop ---
cv = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=42)
scoring = {
    "balanced_accuracy": make_scorer(balanced_accuracy_score),
    "f1": make_scorer(f1_score),
    "accuracy": "accuracy",
    "roc_auc": "roc_auc"
}

cv_results_tuned_model = pd.DataFrame(
    cross_validate(
        tuned_model,
        x,          # your features
        targets,        # your binary labels
        groups=groups,  # same group used in StratifiedGroupKFold
        scoring=scoring,
        cv=cv,
        return_train_score=True,
        return_estimator=True
    )
)

# --- Summary of scores ---
cv_results_tuned_model[
    [col for col in cv_results_tuned_model.columns if "test" in col]
].aggregate(["mean", "std"]).T

In [ ]:
thresholds = [est.best_threshold_ for est in cv_results_tuned_model["estimator"]]

# Mean (sensitive to outliers)
mean_threshold = np.mean(thresholds)

# Median (robust to outliers)
median_threshold = np.median(thresholds)

# Trimmed mean: excludes lowest/highest 10%
from scipy.stats import trim_mean
trimmed_threshold = trim_mean(thresholds, proportiontocut=0.1)

print(f"Mean threshold:     {mean_threshold:.3f}")
print(f"Median threshold:   {median_threshold:.3f}")
print(f"Trimmed threshold:  {trimmed_threshold:.3f}")

In [ ]:
plt.hist(thresholds, bins=10)
plt.axvline(mean_threshold, color='blue', linestyle='--', label='Mean Threshold')
plt.axvline(trimmed_threshold, color='red', linestyle='--', label='Trim Threshold')
plt.axvline(median_threshold, color='green', linestyle='--', label='Median Threshold')
plt.xlabel("Best Thresholds per Fold")
plt.ylabel("Frequency")
plt.title("Distribution of Tuned Thresholds")
plt.legend()
plt.grid(True)
plt.show()

Evaluating Threshold

In [ ]:
best_threshold = 0.04

In [ ]:
def train_evaluate_threshold(X_train, y_train, X_val, y_val, **params):
    model = RandomForestClassifier(random_state=42, n_jobs=-1, **params)
    model.fit(X_train, y_train)

    # Probabilities
    y_prob_train = model.predict_proba(X_train)[:, 1]
    y_prob_val   = model.predict_proba(X_val)[:, 1]

    # Apply threshold
    y_pred_train = (y_prob_train >= best_threshold).astype(int)
    y_pred_val   = (y_prob_val >= best_threshold).astype(int)

    # Metrics
    train_acc = accuracy_score(y_train, y_pred_train)
    val_acc   = accuracy_score(y_val, y_pred_val)
    train_auc = roc_auc_score(y_train, y_prob_train)
    val_auc   = roc_auc_score(y_val, y_prob_val)

    train_precision = precision_score(y_train, y_pred_train, zero_division=0)
    val_precision   = precision_score(y_val, y_pred_val, zero_division=0)
    train_recall    = recall_score(y_train, y_pred_train, zero_division=0)
    val_recall      = recall_score(y_val, y_pred_val, zero_division=0)
    train_f1        = f1_score(y_train, y_pred_train, zero_division=0)
    val_f1          = f1_score(y_val, y_pred_val, zero_division=0)

    avg_precision = average_precision_score(y_val, y_prob_val)

    # Confusion matrix
    cm = confusion_matrix(y_val, y_pred_val)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    print("Threshold Used:", best_threshold)
    print("Train Accuracy: {:.3f}, Validation Accuracy: {:.3f}".format(train_acc, val_acc))
    print("Train AUC: {:.3f},     Validation AUC: {:.3f}".format(train_auc, val_auc))
    print("Train F1: {:.3f},      Validation F1: {:.3f}".format(train_f1, val_f1))
    print("Train Precision: {:.3f}, Validation Precision: {:.3f}".format(train_precision, val_precision))
    print("Train Recall: {:.3f},    Validation Recall: {:.3f}".format(train_recall, val_recall))
    print("Validation AP Score: {:.3f}".format(avg_precision))
    print("Validation Sensitivity (TPR): {:.3f}".format(sensitivity))
    print("Validation Specificity (TNR): {:.3f}".format(specificity))
    print("Confusion Matrix:")
    print(cm)

    return {
        "model": model,
        "train_acc": train_acc,
        "val_acc": val_acc,
        "train_auc": train_auc,
        "val_auc": val_auc,
        "train_f1": train_f1,
        "val_f1": val_f1,
        "train_precision": train_precision,
        "val_precision": val_precision,
        "train_recall": train_recall,
        "val_recall": val_recall,
        "val_ap": avg_precision,
        "val_sensitivity": sensitivity,
        "val_specificity": specificity,
        "conf_matrix": cm
    }


In [ ]:
results = []

for train_idx, val_idx in sgkf.split(x, targets, groups=groups):
    X_tr, y_tr = x.iloc[train_idx], targets.iloc[train_idx]
    X_vl, y_vl = x.iloc[val_idx], targets.iloc[val_idx]

    res = train_evaluate_threshold(
        X_tr, y_tr,
        X_vl, y_vl,
        **best_params_user
    )

    results.append(res)

metrics_df = pd.DataFrame([
    {
        "Train Acc": r["train_acc"],
        "Val Acc": r["val_acc"],
        "Train AUC": r["train_auc"],
        "Val AUC": r["val_auc"],
        "Val F1": r["val_f1"],
        "Val Precision": r["val_precision"],
        "Val Recall": r["val_recall"],
        "Val AP": r["val_ap"],
        "Val Sensitivity": r["val_sensitivity"],
        "Val Specificity": r["val_specificity"]
    }
    for r in results
])

# Show summary statistics
summary_stats = metrics_df.describe().T
print("=== Summary Stats ===")
print(summary_stats)

# Show boxplot of performance across folds
plt.figure(figsize=(12, 6))
sns.boxplot(data=metrics_df)
plt.title("Cross-Validation Performance Metrics")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
for col in ["Val F1", "Val AUC", "Val Precision", "Val Recall"]:
    plt.plot(metrics_df.index, metrics_df[col], marker='o', label=col)

plt.title("Validation Metrics Across Folds")
plt.xlabel("Fold")
plt.ylabel("Score")
plt.legend()
plt.grid(True)
plt.show()
